In [1]:
# ライブラリ読み込み
import pandas as pd

from load_data import load2d
from saver import load_arch
from plotter import plot_samples

Using TensorFlow backend.


In [2]:
model_name = 'model8'
model_arch = 'model/' + model_name + '-arch-5000.json'
model_weights = 'model/' + model_name + '-weights-5000.hdf5'
outpath = 'results/' + model_name + '-5000.csv'

In [3]:
# 定数
SPECIALIST_SETTINGS = [
    dict(
        id='1',
        columns=(
            'left_eye_center_x', 'left_eye_center_y',
            'right_eye_center_x', 'right_eye_center_y',
            ),
        flip_indices=((0, 2), (1, 3)),
        ),

    dict(
        id='2',
        columns=(
            'nose_tip_x', 'nose_tip_y',
            ),
        flip_indices=(),
        ),

    dict(
        id='3',
        columns=(
            'mouth_left_corner_x', 'mouth_left_corner_y',
            'mouth_right_corner_x', 'mouth_right_corner_y',
            'mouth_center_top_lip_x', 'mouth_center_top_lip_y',
            ),
        flip_indices=((0, 2), (1, 3)),
        ),

    dict(
        id='4',
        columns=(
            'mouth_center_bottom_lip_x',
            'mouth_center_bottom_lip_y',
            ),
        flip_indices=(),
        ),

    dict(
        id='5',
        columns=(
            'left_eye_inner_corner_x', 'left_eye_inner_corner_y',
            'right_eye_inner_corner_x', 'right_eye_inner_corner_y',
            'left_eye_outer_corner_x', 'left_eye_outer_corner_y',
            'right_eye_outer_corner_x', 'right_eye_outer_corner_y',
            ),
        flip_indices=((0, 2), (1, 3), (4, 6), (5, 7)),
        ),

    dict(
        id='6',
        columns=(
            'left_eyebrow_inner_end_x', 'left_eyebrow_inner_end_y',
            'right_eyebrow_inner_end_x', 'right_eyebrow_inner_end_y',
            'left_eyebrow_outer_end_x', 'left_eyebrow_outer_end_y',
            'right_eyebrow_outer_end_x', 'right_eyebrow_outer_end_y',
            ),
        flip_indices=((0, 2), (1, 3), (4, 6), (5, 7)),
        ),
    ]

In [8]:
model = model_from_json(model7.to_json()) # アーキテクチャのみを取り出す
model.layers.pop() # 出力層を取り除く
model.outputs = [model.layers[-1].output]
model.layers[-1].outbound_nodes = []
model.add(Dense(len(cols))) # 新しい出力層を追加

In [9]:
sgd = SGD(lr=start, momentum=momentum, nesterov=nesterov)
model.compile(loss=loss_method, optimizer=sgd)


In [ ]:
specialists = OrderedDict()

for setting in SPECIALIST_SETTINGS:
    # 特定のカラムのデータの取得
    cols = setting['columns']
    X, y = load2d(cols=cols)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=42)
    model = load_arch('model/model7-arch-5000.json')
    model.layers.pop() # 出力層を取り除く
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    model.add(Dense(len(cols))) # 新しい出力層を追加

    arch_path = 'model/' + model_name + '-' + setting['id'] + '-arch-' + str(nb_epoch) + '.json'
    save_arch(model, arch_path) # モデルを保存しておく

    # トレーニングの準備
    sgd = SGD(lr=start, momentum=momentum, nesterov=nesterov)
    model.compile(loss=loss_method, optimizer=sgd)
    #plot(model, to_file="model_{}.png".format(cols[0]), show_shapes=True)

    flipgen = FlippedImageDataGenerator()
    flipgen.flip_indices = setting['flip_indices']
    early_stop = EarlyStopping(patience=patience)
    learning_rates = np.linspace(start, stop, nb_epoch)
    change_lr = LearningRateScheduler(lambda epoch: float(learning_rates[epoch]))
    weights_path = 'model/' + model_name + '-' + setting['id'] + '-weights-' + str(nb_epoch) + '.hdf5'
    checkpoint_collback = ModelCheckpoint(filepath = weights_path,
                                          monitor='val_loss',
                                          save_best_only=True,
                                          mode='auto')

    print("Training model for columns {} for {} epochs".format(cols, nb_epoch))


    hist   = model.fit_generator(flipgen.flow(X_train, y_train),
                                 samples_per_epoch=X_train.shape[0],
                                 nb_epoch=nb_epoch,
                                 validation_data=(X_val, y_val),
                                 callbacks=[checkpoint_collback, change_lr, early_stop])
    save_history(hist, model_name)


left_eye_center_x     7039
left_eye_center_y     7039
right_eye_center_x    7036
right_eye_center_y    7036
Image                 7049
dtype: int64
Training model for columns ('left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x', 'right_eye_center_y') for 2 epochs
Epoch 1/2
 384/5626 [=>............................] - ETA: 271s - loss: 0.0361